In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:

url =  "https://www.barcalounger.com/view-all-options/langston-power-lift-recline?attribute_pa_covers=tonya-brown"

In [8]:
req = requests.get(url, headers = {"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"})

In [9]:
req.status_code

202

In [10]:

soup = BeautifulSoup(req.content, "html.parser")

In [13]:
soup.prettify()

'<html>\n <head>\n  <link href="data:;" rel="icon"/>\n  <meta content="0;/.well-known/sgcaptcha/?r=%2Fview-all-options%2Flangston-power-lift-recline%3Fattribute_pa_covers%3Dtonya-brown&amp;y=ipc:41.216.53.25:1738689021.222" http-equiv="refresh"/>\n </head>\n</html>\n'

In [11]:
soup.title.text.strip()

AttributeError: 'NoneType' object has no attribute 'text'

## Web pages html Code saving for selectors checking

In [14]:
# Saving the prettified HTML to a file
with open("product.html", "w", encoding="utf-8") as file:
    file.write(soup.prettify())

## Nav Bar links scraping

In [45]:
menu = soup.find("ul", class_="sub-menu")

In [46]:
categories = menu.find_all("a")

In [47]:
categories = [[item.text.strip(), item.get("href")] for item in categories]

In [48]:
def get_products(link):
    page = requests.get(link)
    products = []
    print("Getting products from : ", link)
    if page.status_code == 200:
        data = BeautifulSoup(page.content, "html.parser")
        products = data.find_all("div", class_ = "sb-col l3 m3 sb-mobile sb-center sb-hover-opacity sb-border-bottom")
        products = [item.find('a').get("href") for item in products if item.find('a')]
    
    if len(products) == 0:
        products = data.find_all('div', class_ = "ut-animated-image-item ut-image-gallery-item ut-animation-done")
        products = [item.find('a').get("href") for item in products if item.find('a')]
        

    if len(products) == 0:
            products = data.find_all('div', class_ = "sb-col l3 m3 sb-mobile sb-center sb-hover-opacity")
            products = [item.find('a').get("href") for item in products if item.find('a')]
        
    print("Products found : ", len(products))
    return products

In [ ]:
def get_products_links():
    url =  "https://smithbrothersfurniture.com/occasional-chairs/"
    req = requests.get(url)
    data = []
    if req.status_code == 200:
        soup = BeautifulSoup(req.content, "html.parser")
        menu = soup.find("ul", class_="sub-menu")
        categories = menu.find_all("a")
        categories = [[item.text.strip(), item.get("href")] for item in categories]
        
        for item in categories:
            category1 = item[0]
            if category1 != "BUILD YOUR OWN":
                products = get_products(item[1])
                for prod_link in products:
                    row = [category1, prod_link]
                    data.append(row)

    return data
        

Getting products from :  https://smithbrothersfurniture.com/new-styles/
Products found :  6
Getting products from :  https://smithbrothersfurniture.com/sofas/
Products found :  35
Getting products from :  https://smithbrothersfurniture.com/sectionals/
Products found :  12
Getting products from :  https://smithbrothersfurniture.com/sb-motion/
Products found :  12
Getting products from :  https://smithbrothersfurniture.com/recliners-tiltbacks/
Products found :  38
Getting products from :  https://smithbrothersfurniture.com/occasional-chairs/
Products found :  42
Getting products from :  https://smithbrothersfurniture.com/occasional-ottomans/
Products found :  11
Getting products from :  https://smithbrothersfurniture.com/sbdiningchairs/
Products found :  18


In [53]:
import csv
# Save data to CSV
csv_filename = "products.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Category", "Product URL"])  # Header
    writer.writerows(data)

print(f"Data saved to {csv_filename}")

Data saved to products.csv


## Product page scraping

In [50]:
sku = soup.find("span", class_ = "sb-style-no")
if sku:
    sku = sku.text.strip()
else:
    sku = ""

In [51]:
sku

''

In [11]:
if sku:
    collection = sku + " " + "Style"
else:
    collection = ""

In [52]:
description = soup.find("p", class_ ="sb-style-desc")
if description:
    description = description.text.strip()
else:
    description = ""

In [53]:
description

''

In [54]:
catalog_pdf = soup.find("div", class_ = "sb-center sb-large")
if catalog_pdf:
    catalog_pdf = catalog_pdf.find("a").get("href")
else:
    catalog_pdf = ""

In [55]:
catalog_pdf

''

In [56]:
main_img = soup.find('img', class_ = "sb-style-details-image")
if main_img:
    main_img = main_img.get("src")
else:
    main_img = ""

In [57]:
main_img

''

In [68]:
product_imgs = soup.find_all("img", class_ = "sb-image sb-hover-opacity sb-wide-25 sb-margin-right")
if product_imgs:
    product_imgs = [item.get("src") for item in product_imgs]
else:
    product_imgs = []

In [70]:
product_imgs = soup.find_all("img", class_ = "sb-image sb-hover-opacity")
if product_imgs:
    product_imgs = [item.get("src") for item in product_imgs]
else:
    product_imgs = []

In [71]:
product_imgs

['https://smithbrothersfurniture.com/ImgGlobal/Styles/public/211-sm-fabric-chair.jpg',
 'https://smithbrothersfurniture.com/ImgGlobal/Styles/public/211-sm-fabric-ottoman.jpg',
 'https://smithbrothersfurniture.com/ImgGlobal/Styles/public/211-sm-leather-sofa.jpg',
 'https://smithbrothersfurniture.com/ImgGlobal/Styles/public/211-sm-leather-chair.jpg',
 'https://smithbrothersfurniture.com/ImgGlobal/Styles/public/211-sm-leather-ottoman.jpg']

In [60]:
specifications = {}
specification_div = soup.find_all("div", class_ = "sb-row sb-mobile sb-margin-left")
if specification_div:
    for item in specification_div:
        divs = item.find_all("div")
        if len(divs) == 2:
            item_key = divs[0].find("strong")
            if item_key:
                item_key = item_key.text.strip().replace(":", "")
            else:
                continue
            item_value = divs[1].find("span")
            if item_value:
                item_value = item_value.text.strip().replace('"', "")
            else:
                continue

            if item_key and item_value:
                specifications[item_key] = item_value

if specifications:
    width = specifications.get("Width", None)
    depth = specifications.get("Depth", None)
    height = specifications.get("Height", None)
    specifications_str = "; ".join([f"{k}: {v}" for k, v in specifications.items()])
else:
    width = ""
    depth = ""
    height = ""
    specifications_str = ""
  


In [61]:
specifications

{}

In [62]:
specifications_str

''

In [75]:
import pandas as pd

# Load the CSV file
file_path = "output/smith_brothers.csv"  # Update with your actual file path
df = pd.read_csv(file_path, dtype=str)

# Remove duplicate rows based on SKU (keeping the first occurrence)
df = df.drop_duplicates(subset=['SKU'], keep='first')

# Save the cleaned data back to a CSV file
df.to_csv("output/smith_brothers_updated.csv", index=False)

print("Duplicate rows removed based on SKU. Updated file saved as 'updated_file.csv'.")


Duplicate rows removed based on SKU. Updated file saved as 'updated_file.csv'.


In [73]:
!pip install pandas

     |████████████████████████████████| 12.4 MB 362 kB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 305 kB/s eta 0:00:01
     |████████████████████████████████| 507 kB 230 kB/s eta 0:00:01
     |████████████████████████████████| 346 kB 350 kB/s eta 0:00:01
